In [1]:
import numpy as np
import pandas as pd
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import string
from nltk.corpus import stopwords
PUNCT_TO_REMOVE = string.punctuation
STOPWORDS = set(stopwords.words('english'))
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from transformers import BertTokenizer , TFBertModel 
AUTO = tf.data.experimental.AUTOTUNE

/kaggle/input/huggingface-bert-variants/bert-large-uncased/bert-large-uncased/config.json
/kaggle/input/huggingface-bert-variants/bert-large-uncased/bert-large-uncased/tokenizer.json
/kaggle/input/huggingface-bert-variants/bert-large-uncased/bert-large-uncased/tf_model.h5
/kaggle/input/huggingface-bert-variants/bert-large-uncased/bert-large-uncased/tokenizer_config.json
/kaggle/input/huggingface-bert-variants/bert-large-uncased/bert-large-uncased/pytorch_model.bin
/kaggle/input/huggingface-bert-variants/bert-large-uncased/bert-large-uncased/vocab.txt
/kaggle/input/huggingface-bert-variants/bert-large-uncased/bert-large-uncased/flax_model.msgpack
/kaggle/input/huggingface-bert-variants/bert-large-uncased/bert-large-uncased/whole-word-masking/._bert_config.json
/kaggle/input/huggingface-bert-variants/bert-large-uncased/bert-large-uncased/whole-word-masking/bert_config.json
/kaggle/input/huggingface-bert-variants/bert-large-uncased/bert-large-uncased/whole-word-masking/pytorch_model.bin
/

2022-11-24 17:31:46.935975: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-24 17:31:46.937068: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-24 17:31:46.937800: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-24 17:31:46.939652: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [18]:
train_df = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/train.csv')
test_df = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/test.csv')
sample_df = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/sample_submission.csv')
bert_path = '../input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased'
print(train_df.shape, test_df.shape, sample_df.shape)

(3911, 8) (3, 2) (3, 7)


In [ ]:
size = train_df.shape[0]
train, validate = int(0.8*size), int(0.2*size)
validate_df = train_df.tail(validate).copy()
train_df = train_df.head(train).copy()
print(train_df.shape, validate_df.shape)

In [12]:
# Merging Train and Test Data
train_size = train_df.shape[0]
test_size = test_df.shape[0]
all_data = pd.concat((train_df, test_df)).reset_index(drop=True)
all_data.drop(['text_id'], axis=1, inplace=True)
print(all_data.shape)

(3914, 7)


In [13]:
lemmatizer = WordNetLemmatizer()

def preprocess(text) : 
    text = text.lower()
    text = re.sub(r'https?://\S+|www\.\S+','', text)
    text = re.sub(r'@[0-9a-zA-Z]*\W+',' ' , text)
    
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'\#', ' ', text)
    text = re.sub(r'\'', ' ', text)
    
    list_text = text.split()
    text = ' '.join(list_text[:512])
    return text

In [14]:
all_data['full_text'] = all_data['full_text'].apply(lambda text : preprocess(text))

In [16]:
train_data = all_data[:train_size].copy()
test_data = all_data[train_size:].copy()

train_data.shape , test_data.shape

((3911, 7), (3, 7))

In [21]:
## Tokenizer
tokenizer = BertTokenizer.from_pretrained(bert_path)

def encode(input_text):
    inputs = tokenizer.batch_encode_plus(input_text,padding='max_length',max_length=MAX_LEN, truncation=True)
    return inputs

In [22]:
train_input = encode(train_data['full_text'].values.tolist())['input_ids']

# Batch size and max sequence length
BATCH_SIZE = 5
MAX_LEN = max(len(x.split()) for x in all_data['full_text'])

train_data_ds = (
    tf.data.Dataset
    .from_tensor_slices((train_input,train_data.drop('full_text', axis = 1)))
    .repeat()
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

In [23]:
testing_input = encode(test_data.full_text.values.tolist())['input_ids']

test_dataset = (
    tf.data.Dataset
    .from_tensor_slices(testing_input)
    .batch(BATCH_SIZE)
)

In [24]:
## Column-wise RMSE
def MCRMSE(y_true, y_pred):
    mcrmse = tf.reduce_mean(tf.square(y_true - y_pred), axis=1)
    return tf.reduce_mean(tf.sqrt(mcrmse), axis=-1, keepdims=True)

In [26]:
def create_model():
    
    ## Pretrained layer
    bert_encoder = TFBertModel.from_pretrained(bert_path)
    input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name="input_word_ids")

    ## Embedding layer
    embedding = bert_encoder(input_word_ids)[0]
    x = tf.keras.layers.GlobalAveragePooling1D()(embedding)
    x = tf.keras.layers.LayerNormalization()(x)
    
    ## 
    output = tf.keras.layers.Dense(6,)(x)
    
    ## Compile model
    model = tf.keras.models.Model(inputs=input_word_ids, outputs=output)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss=MCRMSE
                  , metrics=MCRMSE)

    return model

model= create_model()
model.summary()

Some layers from the model checkpoint at ../input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at ../input/huggingface-bert-variants/bert-base-uncased/bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 512)]             0         
_________________________________________________________________
tf_bert_model (TFBertModel)  TFBaseModelOutputWithPool 109482240 
_________________________________________________________________
global_average_pooling1d (Gl (None, 768)               0         
_________________________________________________________________
layer_normalization (LayerNo (None, 768)               1536      
_________________________________________________________________
dense (Dense)                (None, 6)                 4614      
Total params: 109,488,390
Trainable params: 109,488,390
Non-trainable params: 0
_________________________________________________________________


In [29]:
## Early stopping for overfitting
callback = tf.keras.callbacks.EarlyStopping(monitor='MCRMSE', patience = 2 ,restore_best_weights=True)

## Start Training
history = model.fit(
                    train_data_ds, 
                    steps_per_epoch= train_data.shape[0]//BATCH_SIZE,
                    batch_size = BATCH_SIZE,
                    epochs= 4,
                    verbose = 1,
                    shuffle= True,
                    callbacks=[callback],
                       )

Epoch 1/4


2022-11-24 17:46:15.443566: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-11-24 17:46:20.194967: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


782/782 [==============================] - 329s 401ms/step - loss: 0.5231 - MCRMSE: 0.5231
Epoch 2/4
782/782 [==============================] - 314s 401ms/step - loss: 0.4554 - MCRMSE: 0.4554
Epoch 3/4
782/782 [==============================] - 314s 401ms/step - loss: 0.4295 - MCRMSE: 0.4295
Epoch 4/4
782/782 [==============================] - 314s 401ms/step - loss: 0.4081 - MCRMSE: 0.4081


In [31]:
submission = pd.DataFrame(sample_df)
submission[['cohesion','syntax','vocabulary','phraseology','grammar','conventions']] = model.predict(test_dataset)
submission.to_csv('submission.csv',index=False)
submission

,text_id,cohesion,syntax,vocabulary,phraseology,grammar,conventions
0,0000C359D63E,3.015969,2.679750,3.139230,3.047229,2.709258,2.772438
1,000BAD50D026,2.647339,2.416563,2.762127,2.484760,2.075131,2.788710
2,00367BB2546B,3.835831,3.670458,3.745961,3.721802,3.561491,3.800941


# 